# Multichannel audio source separation by t-ILRMA

In [ ]:
%%shell
git clone https://github.com/tky823/audio_source_separation.git

In [ ]:
%cd "/content/audio_source_separation/egs/bss-example/ilrma"

## 1\. Speech separation

### 1.1 Data preparation
Create multichannel mixtures using the audios of [CMU ARCTIC database](http://www.festvox.org/cmu_arctic/) and impulse responses of [Multi-Channel Impulse Response Database](https://www.iks.rwth-aachen.de/en/research/tools-downloads/databases/multi-channel-impulse-response-database/).

In [ ]:
%%shell
. ./prepare.sh

In [ ]:
import sys
sys.path.append("../../../src")

In [ ]:
import numpy as np
import scipy.signal as ss
import soundfile as sf
import IPython.display as ipd
import matplotlib.pyplot as plt

In [ ]:
from bss.ilrma import tILRMA

In [ ]:
plt.rcParams['figure.dpi'] = 200

Configuration of STFT
- The reverberation time is $T_{60}=160$ [ms] in the impulse response.
- The window length is $4096$ samples (= $256$ [ms]) because of the assumption of rank-1 constraint.
- The hop length is the half of the window length, i.e. $2048$ samples (= $128$ [ms]) .

In [ ]:
fft_size, hop_size = 4096, 2048

### 1.2 2 speakers

In [ ]:
aew_mic3, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic3.wav")
axb_mic3, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic3.wav")
x_mic3 = aew_mic3 + axb_mic3

aew_mic4, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic4.wav")
axb_mic4, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic4.wav")
x_mic4 = aew_mic4 + axb_mic4

x = np.vstack([x_mic3, x_mic4])
n_channels, T = x.shape
n_sources = n_channels

#### Target sources after convolution of impulse response

In [ ]:
display(ipd.Audio(aew_mic3, rate=sr))
display(ipd.Audio(axb_mic3, rate=sr))

#### Mixture

In [ ]:
for idx in range(n_channels):
    display(ipd.Audio(x[idx], rate=sr))

#### Execution of ILRMA

In [ ]:
_, _, X = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)

In [ ]:
np.random.seed(111)
ilrma = tILRMA(n_basis=2, nu=10**3, normalize='projection-back')

In [ ]:
print(ilrma)

In [ ]:
Y = ilrma(X, iteration=50)

In [ ]:
_, y = ss.istft(Y, nperseg=fft_size, noverlap=fft_size-hop_size)
y = y[:,:T]

#### Separated sources

In [ ]:
for idx in range(n_sources):
    display(ipd.Audio(y[idx], rate=sr))

In [ ]:
plt.figure()
plt.plot(ilrma.loss, color='black')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

### 1.3 3 speakers

In [ ]:
aew_mic2, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic2.wav")
axb_mic2, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic2.wav")
bdl_mic2, sr = sf.read("./data/cmu_us_bdl_arctic/trimmed/convolved-16000_deg330-mic2.wav")
x_mic2 = aew_mic2 + axb_mic2 + bdl_mic2

aew_mic4, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic4.wav")
axb_mic4, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic4.wav")
bdl_mic4, sr = sf.read("./data/cmu_us_bdl_arctic/trimmed/convolved-16000_deg330-mic4.wav")
x_mic4 = aew_mic4 + axb_mic4 + bdl_mic4

aew_mic5, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic5.wav")
axb_mic5, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic5.wav")
bdl_mic5, sr = sf.read("./data/cmu_us_bdl_arctic/trimmed/convolved-16000_deg330-mic5.wav")
x_mic5 = aew_mic5 + axb_mic5 + bdl_mic5

x = np.vstack([x_mic2, x_mic4, x_mic5])
n_channels, T = x.shape
n_sources = n_channels

#### Target sources after convolution of impulse response

In [ ]:
display(ipd.Audio(aew_mic2, rate=sr))
display(ipd.Audio(axb_mic2, rate=sr))
display(ipd.Audio(bdl_mic2, rate=sr))

#### Mixture

In [ ]:
for idx in range(n_channels):
    display(ipd.Audio(x[idx], rate=sr))

#### Execution of ILRMA

In [ ]:
_, _, X = ss.stft(x, nperseg=fft_size, noverlap=hop_size)

In [ ]:
np.random.seed(111)
ilrma = tILRMA(n_basis=2, nu=10**3, normalize='projection-back')

In [ ]:
print(ilrma)

In [ ]:
Y = ilrma(X, iteration=100)

In [ ]:
_, y = ss.istft(Y, nperseg=fft_size, noverlap=hop_size)
y = y[:,:T]

#### Separated sources

In [ ]:
for idx in range(n_sources):
    display(ipd.Audio(y[idx], rate=sr))

In [ ]:
plt.figure()
plt.plot(ilrma.loss, color='black')
plt.xlabel('Iteration')
plt.ylabel('Loss')
plt.show()

## 2\. Comparison between Gauss-ILRMA and t-ILRMA

In [ ]:
aew_mic3, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic3.wav")
axb_mic3, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic3.wav")
x_mic3 = aew_mic3 + axb_mic3

aew_mic4, sr = sf.read("./data/cmu_us_aew_arctic/trimmed/convolved-16000_deg60-mic4.wav")
axb_mic4, sr = sf.read("./data/cmu_us_axb_arctic/trimmed/convolved-16000_deg300-mic4.wav")
x_mic4 = aew_mic4 + axb_mic4

x = np.vstack([x_mic3, x_mic4])
n_channels, T = x.shape
n_sources = n_channels

In [ ]:
s = np.vstack([aew_mic3, axb_mic3])
_, _, X = ss.stft(x, nperseg=fft_size, noverlap=fft_size-hop_size)

In [ ]:
%%shell
pip install mir_eval

In [ ]:
from mir_eval.separation import bss_eval_sources

In [ ]:
from algorithm.projection_back import projection_back
from bss.ilrma import GaussILRMA

In [ ]:
def record_sdri(model):
    reference_id = model.reference_id
    s = model.target # Time domain
    X, Y = model.input, model.estimation # Time-frequency domain
    n_sources, T = s.shape

    scale = projection_back(Y, reference=X[reference_id])
    Y = Y * scale[...,np.newaxis] # (n_sources, n_bins, n_frames)
    _, y = ss.istft(Y, nperseg=fft_size, noverlap=fft_size-hop_size)
    y = y[:,:T]

    sdr_estimated, _, _, _ = bss_eval_sources(s, estimated_sources=y)

    if hasattr(model, 'sdr_input'):
        sdr_input = model.sdr_input
    else:
        sdr_input = sdr_estimated
        model.sdr_input = sdr_input
    
    sdri = sdr_estimated - sdr_input
    
    model.sdri.append(sdri.mean())

Gives `record_sdri` as a callback function to record SDR improvements.

In [ ]:
np.random.seed(111)
gauss_ilrma = GaussILRMA(n_basis=2, normalize='projection-back', callbacks=record_sdri)
Y = gauss_ilrma(X, iteration=200, target=s, sdri=[])

In [ ]:
np.random.seed(111)
t_ilrma_1e2 = tILRMA(n_basis=2, nu=1e+2, normalize='projection-back', callbacks=record_sdri)
Y = t_ilrma_1e2(X, iteration=200, target=s, sdri=[])

In [ ]:
np.random.seed(111)
t_ilrma_1e3 = tILRMA(n_basis=2, nu=1e+3, normalize='projection-back', callbacks=record_sdri)
Y = t_ilrma_1e3(X, iteration=200, target=s, sdri=[])

In [ ]:
np.random.seed(111)
t_ilrma_1e4 = tILRMA(n_basis=2, nu=1e+4, normalize='projection-back', callbacks=record_sdri)
Y = t_ilrma_1e4(X, iteration=200, target=s, sdri=[])

In [ ]:
plt.figure()
plt.plot(gauss_ilrma.sdri, color='black', label='Gauss-ILRMA')
plt.plot(t_ilrma_1e2.sdri, color='mediumvioletred', label='t-ILRMA (' + r'$\nu=10^{2}$' + ')')
plt.plot(t_ilrma_1e3.sdri, color='mediumblue', label='t-ILRMA (' + r'$\nu=10^{3}$' + ')')
plt.plot(t_ilrma_1e4.sdri, color='mediumseagreen', label='t-ILRMA (' + r'$\nu=10^{4}$' + ')')
plt.legend()
plt.xlabel('Iteration')
plt.ylabel('SDR improvement')
plt.show()